# Creating Functions

## Learning Objectives

*   Define a function that takes parameters.
*   Return a value from a function.
*   Test and debug a function.
*   Set default values for function parameters.
*   Explain why we should divide programs into small, single-purpose functions.

At this point,
we've written code to draw some interesting features in our bed file data,
loop over all our data files to quickly draw these plots for each of them,
and have Python make decisions based on what it sees in our data.
But, our code is getting pretty long and complicated;
what if we had thousands of datasets,
and didn't want to generate a figure for every single one?

Commenting out the figure-drawing code is a nuisance.
Also, what if we want to use that code again,
on a different dataset or at a different point in our program?
Cutting and pasting it is going to make our code get very long and very repetative,
very quickly.

We'd like a way to package our code so that it is easier to reuse,
and Python provides for this by letting us define things called 'functions' -
a shorthand way of re-executing longer pieces of code.

Let's start by defining a function `fahr_to_kelvin` that converts temperatures from Fahrenheit to Kelvin:

In [1]:
def fahr_to_kelvin(temp):
    return ((temp - 32) * (5/9)) + 273.15

The function definition opens with the word `def`,
which is followed by the name of the function
and a parenthesized list of parameter names.
The [body](reference.html#function-body) of the function --- the
statements that are executed when it runs --- is indented below the definition line,
typically by four spaces.

When we call the function,
the values we pass to it are assigned to those variables
so that we can use them inside the function.
Inside the function,
we use a [return statement](reference.html#return-statement) to send a result back to whoever asked for it.

Let's try running our function.
Calling our own function is no different from calling any other function:

In [2]:
print 'freezing point of water:', fahr_to_kelvin(32)
print 'boiling point of water:', fahr_to_kelvin(212)

freezing point of water: 273.15
boiling point of water: 273.15


We've successfully called the function that we defined,
and we have access to the value that we returned.
Unfortunately, the value returned doesn't look right.
What went wrong?

## Debugging a Function

*Debugging* is when we fix a piece of code
that we know is working incorrectly.
In this case, we know that `fahr_to_kelvin`
is giving us the wrong answer,
so let's find out why.

For big pieces of code,
there are tools called *debuggers* that aid in this process.
Since we just have a short function,
we'll debug by choosing some parameter value,
breaking our function into small parts,
and printing out the value of each part.

In [3]:
# We'll use temp = 212, the boiling point of water, which was incorrect
print "212 - 32:", 212 - 32

212 - 32: 180


In [4]:
print "(212 - 32) * (5/9):", (212 - 32) * (5/9)

(212 - 32) * (5/9): 0


Aha! The problem comes when we multiply by `5/9`.
This is because `5/9` is actually 0.

In [5]:
5/9

0

Computers store numbers in one of two ways:
as [integers](reference.html#integer)
or as [floating-point numbers](reference.html#floating-point-number) (or floats).
The first are the numbers we usually count with;
the second have fractional parts.
Addition, subtraction and multiplication work on both as we'd expect,
but division works differently.
If we divide one integer by another,
we get the quotient without the remainder:

In [6]:
print '10/3 is:', 10/3

10/3 is: 3


If either part of the division is a float,
on the other hand,
the computer creates a floating-point answer:

In [7]:
print '10.0/3 is:', 10.0/3


10.0/3 is: 3.33333333333


The computer does this for historical reasons:
integer operations were much faster on early machines,
and this behavior is actually useful in a lot of situations.
It's still confusing,
though,
so Python 3 produces a floating-point answer when dividing integers if it needs to.
We're still using Python 2.7 in this class,
though,
so if we want `5/9` to give us the right answer,
we have to write it as `5.0/9`, `5/9.0`, or some other variation.

Another way to create a floating-point answer
is to explicitly tell the computer that you desire one.
This is achieved by [casting](reference.html#typecast) one of the numbers:

In [8]:
print 'float(10)/3 is:', float(10)/3

float(10)/3 is: 3.33333333333


The advantage to this method is it can be used with existing variables.
Let's take a look:

In [9]:
a = 10
b = 3
print 'a/b is:', a/b
print 'float(a)/b is:', float(a)/b


a/b is: 3
float(a)/b is: 3.33333333333


Let's fix our `fahr_to_kelvin` function with this new knowledge:

In [10]:
def fahr_to_kelvin(temp):
    return ((temp - 32) * (5.0/9.0)) + 273.15

print 'freezing point of water:', fahr_to_kelvin(32)
print 'boiling point of water:', fahr_to_kelvin(212)

freezing point of water: 273.15
boiling point of water: 373.15


## Composing Functions

Now that we've seen how to turn Fahrenheit into Kelvin,
it's easy to turn Kelvin into Celsius:

In [11]:
def kelvin_to_celsius(temp):
    return temp - 273.15

print 'absolute zero in Celsius:', kelvin_to_celsius(0.0)


absolute zero in Celsius: -273.15


What about converting Fahrenheit to Celsius?
We could write out the formula,
but we don't need to.
Instead,
we can [compose](reference.html#function-composition) the two functions we have already created:

In [12]:
def fahr_to_celsius(temp):
    temp_k = fahr_to_kelvin(temp)
    result = kelvin_to_celsius(temp_k)
    return result

print 'freezing point of water in Celsius:', fahr_to_celsius(32.0)

freezing point of water in Celsius: 0.0


This is our first taste of how larger programs are built:
we define basic operations,
then combine them in ever-large chunks to get the effect we want.
Real-life functions will usually be larger than the ones shown here --- typically half a dozen to a few dozen lines --- but
they shouldn't ever be much longer than that,
or the next person who reads it won't be able to understand what's going on.

## Tidying up

Now that we know how to wrap bits of code up in functions,
we can make our inflammation analyasis easier to read and easier to reuse.
First, let's make an `analyze` function that generates our plots:

In [13]:
import pandas

In [14]:
def analyze(filename):
    data = pandas.read_table(filename, header=None)
    data.columns = ['chrom','chromStart','chromEnd','name','score','strand','level','signif','score2']
    data.groupby('chrom').mean().plot(kind='bar',y='score', ylim=[0,250], title=f, figsize=(10,5))

and another function called `detect_problems` that checks for those systematics
we noticed:

In [15]:
def detect_problems(filename):
    data = pandas.read_table(filename, header=None)
    data.columns = ['chrom','chromStart','chromEnd','name','score','strand','level','signif','score2']
    if data['score2'].min() < 1 and data['score'].min() > 0:
        print 'Suspicious data!'
    elif data.loc[data['chrom'] == 'chrM']['score'].mean() > 200:
        print 'High scores on chrM!'
    else:
        print 'Seems OK!'   

Notice that rather than jumbling this code together in one giant `for` loop,
we can now read and reuse both ideas separately.
We can reproduce the previous analysis with a much simpler `for` loop:

In [16]:
import glob
filenames = glob.glob('data/*.bed')
for f in filenames[:3]:
    print f
    analyze(f)
    detect_problems(f)


data/A549.bed
High scores on chrM!


By giving our functions human-readable names,
we can more easily read and understand what is happening in the `for` loop.
Even better, if at some later date we want to use either of those pieces of code again,
we can do so in a single line.

## Testing and Documenting

Once we start putting things in functions so that we can re-use them,
we need to start testing that those functions are working correctly.
To see how to do this,

let's write a function to center a column of data (a series) around a particular value:

In [17]:
def center_series(series, desired):
    return (series - series.mean()) + desired

We could test this on our actual data,
but since we don't know what the values ought to be,
it will be hard to tell if the result was correct.
Instead,
let's use Pandas to create a Series of 0's
and then center that around 3:

In [18]:
series = pandas.Series([0,0,0,0], name='zeroes')
center_series(series, 3)

0    3
1    3
2    3
3    3
Name: zeroes, dtype: float64

That looks right,
so let's try `center_series` on the **score** column from our real data:

In [35]:
import pandas
data = pandas.read_table('data/A549.bed', header=None)
cols = ['chrom','chromStart','chromEnd','name','score','strand','level','signif','score2']
data.columns = cols
score = data['score']
center_series(score, 0)


0    -64.395554
1    -61.395554
2     -8.395554
3    -60.395554
4    933.604446
Name: score, dtype: float64

It's hard to tell from the default output whether the result is correct,
but there are a few simple tests that will reassure us:

In [20]:
print 'original min, mean, and max are:', score.min(), score.mean(), score.max()
centered = center_series(score, 0)
print 'min, mean, and and max of centered data are:', centered.min(), centered.mean(), centered.max()

original min, mean, and max are: 0 66.395553809 1000
min, mean, and and max of centered data are: -66.395553809 4.88911480585e-14 933.604446191


That seems almost right:
the original mean was about 66.4
so the lower bound from zero is now about -66.4.
The mean of the centered data isn't quite zero --- we'll explore why not in the challenges --- but it's pretty close.
We can even go further and check that the standard deviation hasn't changed:

In [21]:
print 'std dev before and after:', score.std(), centered.std()

std dev before and after: 182.96723548 182.96723548


Those values look the same,
but we probably wouldn't notice if they were different in the sixth decimal place.
Let's do this instead:

In [22]:
print 'difference in standard deviations before and after:', score.std() - centered.std()

difference in standard deviations before and after: 0.0


We have one more task, though:
we should write some [documentation](reference.html#documentation) for our function
to remind ourselves later what it's for and how to use it.

The usual way to put documentation in software is to add [comments](reference.html#comment) like this:

In [23]:
# center(series, desired): return a new series containing the original data centered around the desired value.
def center_series(series, desired):
    return (series - series.mean()) + desired


There's a better way, though.
If the first thing in a function is a string that isn't assigned to a variable,
that string is attached to the function as its documentation:

In [24]:
def center_series(series, desired):
    '''Return a new series containing the original data centered around the desired value.'''
    return (series - series.mean()) + desired


This is better because we can now ask Python's built-in help system to show us the documentation for the function:

In [25]:
help(center_series)

Help on function center_series in module __main__:

center_series(series, desired)
    Return a new series containing the original data centered around the desired value.



A string like this is called a [docstring](reference.html#docstring).
We don't need to use triple quotes when we write one,
but if we do,
we can break the string across multiple lines:

In [26]:
def center_series(series, desired):
    '''Return a new series containing the original data centered around the desired value.
    Example: center_series(pandas.Series(data=[1, 2, 3]), 0) => [-1, 0, 1]'''
    return (series - series.mean()) + desired


In [27]:
pandas.Series(data=[0,0,0,0],name='zeroes')

0    0
1    0
2    0
3    0
Name: zeroes, dtype: int64

## Defining Defaults

We have passed parameters to functions in two ways:
directly, as in `type(data)`,

and by name, as in `pandas.read_table('data/A549.bed', header=None)`
But if we try to use this without the `header=`:

In [28]:
pandas.read_table('data/A549.bed', None)

/Users/dcl9/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  ParserWarning)


,chr10,100023307,100023327,chr10:100023307-100023327(unannotated),2,-,2.34,0.205,3
0,chr10,100082171,100082191,chr10:100082171-100082191(unannotated),5,-,4.69,0.138,6
1,chr10,100082171,100082191,chr10:100082171-100082191(unannotated),58,+,57.83,0.106,74
2,chr10,100152747,100152764,chr10:100152747-100152764(unannotated),6,+,5.52,0.120,6
3,chr10,100155026,100155049,chr10:100155026-100155049(miRNA|processed_tran...,1000,-,2744.14,0.007,4038
4,chr10,100206760,100206783,chr10:100206760-100206783(unannotated),7,+,7.48,0.120,11
5,chr10,100246530,100246555,chr10:100246530-100246555(unannotated),6,+,5.63,0.138,9
6,chr10,1008275,1008291,chr10:1008275-1008291(unannotated),9,+,8.79,0.106,9
7,chr10,101128603,101128622,chr10:101128603-101128622(unannotated),5,+,4.94,0.138,6
8,chr10,101155393,101155409,chr10:101155393-101155409(unannotated),13,-,12.70,0.081,13
9,chr10,101157821,101157840,chr10:101157821-101157840(unannotated),49,+,49.36,0.205,60


The data does not load correctly.

To understand what's going on,
and make our own functions easier to use,
let's re-define our `center_series` function like this:

In [29]:
def center_series(series, desired=0.0):
    '''Return a new series containing the original data centered around the desired value (0 by default).
    Example: center_series(pandas.Series(data=[1, 2, 3]), 0) => [-1, 0, 1]'''
    return (series - series.mean()) + desired


The key change is that the second parameter is now written `desired=0.0` instead of just `desired`.
If we call the function with two arguments,
it works as it did before:

In [30]:
test_series = pandas.Series([0,0])
print center_series(test_series, 3)

0    3
1    3
dtype: float64


But we can also now call it with just one parameter,
in which case `desired` is automatically assigned the [default value](reference.html#default-value) of 0.0:

In [31]:
another_series = pandas.Series([5,5])
print 'series before centering:'
print another_series
print
print 'centered series:'
print center_series(another_series)


series before centering:
0    5
1    5
dtype: int64

centered series:
0    0
1    0
dtype: float64


This is handy:
if we usually want a function to work one way,
but occasionally need it to do something else,
we can allow people to pass a parameter when they need to
but provide a default to make the normal case easier.
The example below shows how Python matches values to parameters:

In [32]:
def display(a=1, b=2, c=3):
    print 'a:', a, 'b:', b, 'c:', c

print 'no parameters:'
display()
print 'one parameter:'
display(55)
print 'two parameters:'
display(55, 66)

no parameters:
a: 1 b: 2 c: 3
one parameter:
a: 55 b: 2 c: 3
two parameters:
a: 55 b: 66 c: 3


As this example shows,
parameters are matched up from left to right,
and any that haven't been given a value explicitly get their default value.
We can override this behavior by naming the value as we pass it in:

In [33]:
print 'only setting the value of c'
display(c=77)

only setting the value of c
a: 1 b: 2 c: 77


With that in hand,
let's look at the help for `pandas.read_table`:

In [34]:
help(pandas.read_table)

Help on function read_table in module pandas.io.parsers:

read_table(filepath_or_buffer, sep='\t', dialect=None, compression='infer', doublequote=True, escapechar=None, quotechar='"', quoting=0, skipinitialspace=False, lineterminator=None, header='infer', index_col=None, names=None, prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None, na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None, dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False, na_filter=True, compact_ints=False, use_unsigned=False, low_memory=True, buffer_lines=None, warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None, decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None, memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None, verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False, infer_datetime_format=Fa

There's a lot of information here,
but the most important part is the first bit:
```python
read_table(filepath_or_buffer, sep='\t', dialect=None, compression='infer', doublequote=True, escapechar=None, quotechar='"', quoting=0, skipinitialspace=False, lineterminator=None, header='infer'...
```

This tells us that `read_table` has one parameter called `filepath_or_buffer` that doesn't have a default value,
and many others that do.
If we call the function like this:

```python
pandas.read_table('data/A549.bed', None)
```

then the filename is assigned to `filepath_or_buffer` (which is what we want). Our second parameter, `None`, is assigned to `sep` rather than `header`. This is because `sep` is the second parameter in the function definition. So when we just provide the value in that second parameter, we're not setting `header` to anything. The help explains that this defaults to `0` in our case, which means the first row of data will be treated as headers.

If we want `None` to be assigned to the variable `header`, we **do** have to provide `header=None` for the second parameter, because `header` is not the second parameter

## Combining strings

"Adding" two strings produces their concatenation:
`'a' + 'b'` is `'ab'`.
Write a function called `fence` that takes two parameters called `original` and `wrapper`
and returns a new string that has the wrapper character at the beginning and end of the original.
A call to your function should look like this:

~~~ {.python}
print fence('name', '*')
*name*
~~~

## Selecting characters from strings

If the variable `s` refers to a string,
then `s[0]` is the string's first character
and `s[-1]` is its last.
Write a function called `outer`
that returns a string made up of just the first and last characters of its input.
A call to your function should look like this:

~~~ {.python}
print outer('helium')
hm
~~~

## Variables inside and outside functions

What does the following piece of code display when run - and why?

~~~ {.python}
f = 0
k = 0

def f2k(f):
 k = ((f-32)*(5.0/9.0)) + 273.15
 return k

f2k(8)
f2k(41)
f2k(32)

print k
~~~